In [3]:
import numpy as np
import mne  # For EEG preprocessing
import pandas as pd  # For CSV data handling
from scipy.signal import welch
from sklearn.preprocessing import MinMaxScaler
import joblib  # For loading a pre-trained ML model
from muselsl import stream, record  # For real-time EEG data from Muse S

# Retrieve real-time EEG data from Muse S

# Compute emotion scores from PSD


In [4]:

# Load EEG data from a FIF file
def load_eeg_data(file_path):
    raw = mne.io.read_raw_fif(file_path, preload=True)
    raw.filter(1, 50)  # Bandpass filter (1-50 Hz)
    return raw

# Load EEG data from a CSV file
def load_csv_data(file_path):
    df = pd.read_csv(file_path)
    return df


In [5]:
def get_eeg_from_muse(duration=10, filename="muse_eeg.csv"):
    print("Starting Muse stream...")
    stream()
    print("Recording EEG data...")
    record(duration=duration, filename=filename)
    print("EEG data saved to", filename)
    return load_csv_data(filename)

# Extract power spectral density features
def extract_psd_features(df, channels=['TP9', 'TP10', 'AF7', 'AF8']):
    psd_features = {}
    for ch in channels:
        if ch in df.columns:
            data = df[ch].values
            freqs, psd = welch(data, fs=256, nperseg=256)
            psd_features[ch] = {
                'delta': np.mean(psd[(freqs >= 0.5) & (freqs < 4)]),
                'theta': np.mean(psd[(freqs >= 4) & (freqs < 8)]),
                'alpha': np.mean(psd[(freqs >= 8) & (freqs < 12)]),
                'beta': np.mean(psd[(freqs >= 12) & (freqs < 30)]),
                'gamma': np.mean(psd[(freqs >= 30) & (freqs < 50)])
            }
    return psd_features


In [6]:
def compute_emotion_scores(psd_features):
    features = np.array([[
        psd_features['AF7']['beta'], psd_features['AF8']['beta'],  # Anxiety
        psd_features['TP9']['alpha'], psd_features['TP10']['alpha'],  # Calmness
        psd_features['AF7']['theta'], psd_features['AF8']['theta'],  # Sadness
        psd_features['AF7']['gamma'], psd_features['AF8']['gamma'],  # Anger
        psd_features['TP9']['gamma'], psd_features['TP10']['gamma']   # Fear
    ]])
    
    # Load pre-trained model
    model = joblib.load("emotion_model.pkl")
    scores = model.predict(features)[0]
    
    # Scale to 1-10
    scaler = MinMaxScaler(feature_range=(1, 10))
    scores = scaler.fit_transform(scores.reshape(-1, 1)).flatten()
    
    return {
        'Anxiety': scores[0],
        'Calmness': scores[1],
        'Sadness': scores[2],
        'Anger': scores[3],
        'Fear': scores[4]
    }

# Example usage
def main():
    df = get_eeg_from_muse()
    psd_features = extract_psd_features(df)
    emotion_scores = compute_emotion_scores(psd_features)
    print("Emotion Scores:", emotion_scores)

if __name__ == "__main__":
    main()


Starting Muse stream...


TypeError: stream() missing 1 required positional argument: 'address'

In [17]:
import numpy as np
import mne  # For EEG preprocessing
import pandas as pd  # For CSV data handling
from scipy.signal import welch
from sklearn.preprocessing import MinMaxScaler
import joblib  # For loading a pre-trained ML model
from pylsl import StreamInlet, resolve_byprop  # For real-time EEG data from Muse S

# Load EEG data from a FIF file
def load_eeg_data(file_path):
    raw = mne.io.read_raw_fif(file_path, preload=True)
    raw.filter(1, 50)  # Bandpass filter (1-50 Hz)
    return raw

# Load EEG data from a CSV file
def load_csv_data(file_path):
    df = pd.read_csv(file_path)
    return df

def get_eeg_from_muse(duration=10):
    print("Resolving EEG stream...")
    streams = resolve_byprop('type', 'EEG', timeout=5)
    if not streams:
        raise RuntimeError("No EEG stream found.")
    
    inlet = StreamInlet(streams[0])
    print("Connected to EEG stream.")
    
    eeg_data = []
    for _ in range(int(duration * 256)):  # Assuming 256 Hz sampling rate
        sample, _ = inlet.pull_sample()
        eeg_data.append(sample)
    
    # Muse S has 5 channels: TP9, AF7, AF8, TP10, Right AUX
    df = pd.DataFrame(eeg_data, columns=['TP9', 'AF7', 'AF8', 'TP10', 'Right AUX'])
    print("EEG data acquired.")
    return df


# Extract power spectral density features
def extract_psd_features(df, channels=['TP9', 'TP10', 'AF7', 'AF8']):
    psd_features = {}
    for ch in channels:
        if ch in df.columns:
            data = df[ch].values
            freqs, psd = welch(data, fs=256, nperseg=256)
            psd_features[ch] = {
                'delta': np.mean(psd[(freqs >= 0.5) & (freqs < 4)]),
                'theta': np.mean(psd[(freqs >= 4) & (freqs < 8)]),
                'alpha': np.mean(psd[(freqs >= 8) & (freqs < 12)]),
                'beta': np.mean(psd[(freqs >= 12) & (freqs < 30)]),
                'gamma': np.mean(psd[(freqs >= 30) & (freqs < 50)])
            }
    return psd_features

# Compute emotion scores from PSD
def compute_emotion_scores(psd_features):
    # Define basic emotion scoring system based on PSD bands
    # Higher beta -> anxiety, higher alpha -> calmness, etc.
    emotion_scores = {}

    # Anxiety: higher beta activity (14-30 Hz) in AF7, AF8
    anxiety_score = ((psd_features['AF7']['beta'] + psd_features['AF8']['beta']) / 2)
    if anxiety_score >= 100:
        anxiety_score = 100
    emotion_scores['Anxiety'] = anxiety_score//10 # Scale to 1-10
    
    # Calmness: higher alpha activity (8-12 Hz) in TP9, TP10
    calmness_score = (psd_features['TP9']['alpha'] + psd_features['TP10']['alpha']) / 2
    if calmness_score >= 100:
        calmness_score = 100
    emotion_scores['Calmness'] = calmness_score //10  # Scale to 1-10

    # Sadness: higher theta activity (4-8 Hz) in AF7, AF8
    sadness_score = (psd_features['AF7']['theta'] + psd_features['AF8']['theta']) / 2
    if sadness_score >= 100:
        sadness_score = 100
    emotion_scores['Sadness'] = sadness_score//10 # Scale to 1-10

    # Anger: higher gamma activity (30-50 Hz) in AF7, AF8
    anger_score = (psd_features['AF7']['gamma'] + psd_features['AF8']['gamma']) / 2
    if anger_score >= 100:
        anger_score = 100
    emotion_scores['Anger'] = anger_score//10  # Scale to 1-10

    # Fear: higher gamma activity (30-50 Hz) in TP9, TP10
    fear_score = (psd_features['TP9']['gamma'] + psd_features['TP10']['gamma']) / 2
    if fear_score >= 100:
        fear_score = 100
    emotion_scores['Fear'] =  fear_score//10
    print(anxiety_score,calmness_score,sadness_score,anger_score,fear_score)

    return emotion_scores

# # Example usage
# psd_features = {
#     'AF7': {'beta': 0.4, 'alpha': 0.2, 'theta': 0.1, 'gamma': 0.8},
#     'AF8': {'beta': 0.5, 'alpha': 0.3, 'theta': 0.2, 'gamma': 0.9},
#     'TP9': {'beta': 0.3, 'alpha': 0.7, 'theta': 0.1, 'gamma': 0.4},
#     'TP10': {'beta': 0.2, 'alpha': 0.6, 'theta': 0.1, 'gamma': 0.3},
# }

# emotion_scores = compute_emotion_scores(psd_features)
# print("Emotion Scores:", emotion_scores)


# Example usage
def main():
    df = get_eeg_from_muse()
    psd_features = extract_psd_features(df)
    emotion_scores = compute_emotion_scores(psd_features)
    print("Emotion Scores:", emotion_scores)

if __name__ == "__main__":
    main()


Resolving EEG stream...
Connected to EEG stream.
EEG data acquired.
91.25178274530872 52.094577685264966 100 37.807577720456955 13.940281579062535
Emotion Scores: {'Anxiety': 9.0, 'Calmness': 5.0, 'Sadness': 10, 'Anger': 3.0, 'Fear': 1.0}
